# This file was made to test different algorithms for finding Sudoku solutions. 
**The algorithms are:**
- Backtracking with heuristics
- DLX
- SAT Solvers
- Linear Programming
- DLX with different heuristics

In [1]:
# First, we want to import the necessary libraries
import numpy as np
import pandas as pd

In [ ]:
# Then we want to read the data from the csv file
sudokus = pd.read_csv(r"C:\Users\etien\Telecom\sudoku.csv\sudoku.csv")

# We only want to keep 3000 random sudoku puzzles
sudokus = sudokus.sample(n=3000)